<a href="https://colab.research.google.com/github/naveenk-DS/Text_To_Image/blob/main/Text_to_Image_App_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers transformers accelerate safetensors torch torchvision torchaudio --upgrade

In [ ]:
! pip install customtkinter
# 1. Import libraries
# -----------------------------
from pyvirtualdisplay import Display  # Virtual screen for Tkinter in Colab
import tkinter as tk                  # Tkinter GUI
import customtkinter as ctk           # Modern Tkinter styling
from PIL import ImageTk               # Pillow for images

import torch
from torch import autocast
from diffusers import StableDiffusionPipeline

In [ ]:
# -----------------------------
# 2. Start virtual display (needed only in Colab, skip in VSCode)
# -----------------------------
display = Display(visible=0, size=(800, 600))
display.start()

In [ ]:
# 3. Create main Tkinter app
# -----------------------------
app = tk.Tk()                          # Main window
app.geometry("532x632")                # Window size
app.title("Text_to_Image")             # Window title
ctk.set_appearance_mode("dark")        # Dark mode UI

In [ ]:
# 4. Input text box
# -----------------------------
prompt = ctk.CTkEntry(
    app, height=40, width=512,
    font=("Arial", 20),
    text_color="black",
    fg_color="white"
)
prompt.place(x=10, y=10)

In [ ]:
# 5. Output image label
# -----------------------------
lmain = ctk.CTkLabel(app, height=512, width=512)
lmain.place(x=10, y=110)

In [ ]:
# 6. Load Stable Diffusion model
# -----------------------------
#import os
#from huggingface_hub import notebook_login
! pip install authtoken

#from authtoken import auth_token      # Your HF token

auth_token = "hf_your_huggingface_token_here"  # Replace with your actual token


modelid = "CompVis/stable-diffusion-v1-4"
device = "cpu"   # Colab CPU (change to "cuda" if GPU available)

pipe = StableDiffusionPipeline.from_pretrained(
    modelid,
    revision="fp16",
    torch_dtype=torch.float32,
    use_auth_token=auth_token
)
pipe.to(device)

print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

ERROR: Could not find a version that satisfies the requirement authtoken (from versions: none)
ERROR: No matching distribution found for authtoken


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/diffusers/pipelines/pipeline_loading_utils.py:325: FutureWarning: You are loading the variant fp16 from CompVis/stable-diffusion-v1-4 via `revision='fp16'`. This behavior is deprecated and will be removed in diffusers v1. One should use `variant='fp16'` instead. However, it appears that CompVis/stable-diffusion-v1-4 currently does not have the required variant filenames in the 'main' branch. 
 The Diffusers team and community would be ver

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
/usr/local/lib/python3.11/dist-packages/transformers/models/clip/feature_extraction_clip.py:30

CUDA Available: True
GPU Name: Tesla T4


In [ ]:
# 7. Generate function
# -----------------------------
def generate():
    # Use autocast for mixed precision (only works if GPU available)
    with autocast(device) if device == "cuda" else torch.no_grad():
        result = pipe(prompt.get(), guidance_scale=8.5)
        image = result["sample"][0]

    # Save and show image
    image.save("generateimage.png")
    img = ImageTk.PhotoImage(image)
    lmain.configure(image=img)
    lmain.image = img  # keep reference


In [ ]:
# 8. Button for generation
# -----------------------------
trigger = ctk.CTkButton(
    app, height=40, width=120,
    font=("Arial", 20),
    text_color="black",
    fg_color="blue",
    command=generate
)
trigger.configure(text="Generate")
trigger.place(x=206, y=60)
